# **Redes Neurais e Aprendizado Profundo**
**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook apresenta um exemplo de uma rede neural profunda com mais de uma camada para um problema de classificação.


# **Saúde Fetal**

As Cardiotocografias (CTGs) são opções simples e de baixo custo para avaliar a saúde fetal, permitindo que os profissionais de saúde atuem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais.

Este conjunto de dados contém 2126 registros de características extraídas de exames de Cardiotocografias, que foram então classificados por três obstetras especialistas em 3 classes:

- Normal
- Suspeito
- Patológico

# Clonando o conjunto de dados

In [ ]:
!rm -r lectures-cdas-2023
!git clone https://github.com/renansantosmendes/lectures-cdas-2023.git

rm: cannot remove 'lectures-cdas-2023': No such file or directory
Cloning into 'lectures-cdas-2023'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 0), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (12/12), 174.34 KiB | 1.44 MiB/s, done.


# Instalando pacotes

In [ ]:
!pip install mlflow dagshub gitpython -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.2

# 1 - Importando os módulos necessários

In [ ]:
import os
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2

import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definindo funções adicionais

In [ ]:
import os
import random
import numpy as np
import random as python_random

def reset_seeds():
   os.environ['PYTHONHASHSEED']=str(42)
   tf.random.set_seed(42)
   np.random.seed(42)
   random.seed(42)

# 2 - Fazendo a leitura do dataset e atribuindo às respectivas variáveis

In [ ]:
data = pd.read_csv(os.path.join('lectures-cdas-2023', 'fetal_health_reduced.csv'))

# Dando uma leve olhada nos dados

In [ ]:
data.head()

,severe_decelerations,accelerations,fetal_movement,uterine_contractions,fetal_health
0,0.0,0.0,0.0,0.0,2.0
1,0.0,6.0,0.0,6.0,1.0
2,0.0,3.0,0.0,8.0,1.0
3,0.0,3.0,0.0,8.0,1.0
4,0.0,7.0,0.0,8.0,1.0


In [ ]:
data.fetal_health.value_counts()

1.0    1655
2.0     295
3.0     176
Name: fetal_health, dtype: int64

# 3 - Preparando o dado antes de iniciar o treino do modelo

In [ ]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"]

columns_names = list(X.columns)
scaler = preprocessing.StandardScaler()
X_df = scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)

y_train = y_train -1
y_test = y_test - 1

In [ ]:
y_train.value_counts()

0.0    1159
1.0     194
2.0     135
Name: fetal_health, dtype: int64

In [ ]:
X_train

,severe_decelerations,accelerations,fetal_movement,uterine_contractions
1718,-0.057476,1.247640,-0.203210,-0.803434
857,-0.057476,-0.822388,-0.203210,-0.124404
1075,-0.057476,-0.304881,-0.203210,0.894142
371,-0.057476,-0.822388,-0.010304,-0.803434
222,-0.057476,-0.304881,0.075432,-1.482465
...,...,...,...,...
1638,-0.057476,1.506393,-0.181776,-0.124404
1095,-0.057476,2.282654,-0.203210,-0.803434
1130,-0.057476,0.471379,-0.203210,-0.124404
1294,-0.057476,-0.046128,-0.203210,1.233657


# 4 - Criando o modelo e adicionando as camadas

In [ ]:
reset_seeds()
model = Sequential()
model.add(InputLayer(input_shape=(4,)))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 3)                 33        
                                                                 
Total params: 303 (1.18 KB)
Trainable params: 303 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 5 - Compilando o modelo


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 6 - Executando o treino do modelo

In [ ]:
model.fit(x=X_train,
          y=y_train,
          epochs=50)

Epoch 1/50
47/47 [==============================] - 1s 3ms/step - loss: 1.0225 - accuracy: 0.6707
Epoch 2/50
47/47 [==============================] - 0s 3ms/step - loss: 0.8690 - accuracy: 0.7789
Epoch 3/50
47/47 [==============================] - 0s 4ms/step - loss: 0.7349 - accuracy: 0.7789
Epoch 4/50
47/47 [==============================] - 0s 3ms/step - loss: 0.6461 - accuracy: 0.7789
Epoch 5/50
47/47 [==============================] - 0s 4ms/step - loss: 0.5994 - accuracy: 0.7789
Epoch 6/50
47/47 [==============================] - 0s 3ms/step - loss: 0.5700 - accuracy: 0.7789
Epoch 7/50
47/47 [==============================] - 0s 2ms/step - loss: 0.5479 - accuracy: 0.7897
Epoch 8/50
47/47 [==============================] - 0s 7ms/step - loss: 0.5296 - accuracy: 0.8038
Epoch 9/50
47/47 [==============================] - 0s 5ms/step - loss: 0.5137 - accuracy: 0.8044
Epoch 10/50
47/47 [==============================] - 0s 2ms/step - loss: 0.5048 - accuracy: 0.8044
Epoch 11/50
47/47 [